In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import matplotlib.pyplot as plt

In [2]:
filename='processed\SAP_Test_Data_BKPF Table.txt_processed'
#filename='processed\SAP_Test_Data_GLT0 Table.txt_processed'
#filename='processed\SAP_Test_Data_BSEG Table.txt_processed'

In [3]:
df= pd.read_csv(filename,sep='|',dtype=str,low_memory=False)
df_string= pd.read_csv(filename,sep='|',dtype=str,low_memory=False)
df.head(5)

,Buchungskreis,Belegnummer,Geschäftsjahr,Belegart,Belegdatum,Buchungsdatum,Buchungsperiode,Erfassungsdatum,Erfassungsuhrzeit,Geändert am,...,Übergreifende Nr,Referenz,Jahr,Belegkopftext,Währung,Umrechnungskurs,Konzernwährung,Konzernkurs,Belegstatus,Dokumentenname im Archivsystem
0,1710,100002131,2016,SA,2016-01-01 00:00:00,2016-01-01 00:00:00,1,2016-12-15 00:00:00,1900-01-01 11:07:34,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,NaN,NaN,NaN
1,1710,100002132,2016,SA,2016-01-01 00:00:00,2016-01-01 00:00:00,1,2016-12-15 00:00:00,1900-01-01 11:07:34,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,NaN,NaN,NaN
2,1710,100002133,2016,SA,2016-01-01 00:00:00,2016-01-01 00:00:00,1,2016-12-15 00:00:00,1900-01-01 11:07:34,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,NaN,NaN,NaN
3,1710,100002134,2016,SA,2016-01-01 00:00:00,2016-01-01 00:00:00,1,2016-12-15 00:00:00,1900-01-01 11:07:34,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,NaN,NaN,NaN
4,1710,100002135,2016,SA,2016-01-01 00:00:00,2016-01-01 00:00:00,1,2016-12-15 00:00:00,1900-01-01 11:07:35,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,NaN,NaN,NaN


In [4]:
def convert_int(convert,df):
    pbjekts=pd.DataFrame(df.dtypes,columns=['types'])
    for x in pbjekts[pbjekts['types']=='object'].index.to_list():
        try:
            df[x]=df[x].astype(convert)
        except:
            pass

In [5]:
def date_check(df,var):
    col=[s for s in df.columns.to_list() if any([x.lower for x in var if x.lower() in s.lower()])]
    for x in col:
        if df[x].str.len().mean()==4:
            #print('ja')
            col.remove(x)
    return col

In [6]:
var=['jahr','datum','Dat']
col=date_check(df,var)
df[col]=df.loc[:,col].apply(pd.to_datetime)
var2=['zeit','uhr']
col2=date_check(df,var2)
df[col2]=df.loc[:,col2].apply(pd.to_datetime)
for x in col2:
    df[x]=df[x].dt.time

In [7]:
pbjekts=pd.DataFrame(df.dtypes,columns=['types'])
list=pbjekts[pbjekts['types']=='object'].index.to_list()
zeiten=['jahr','datum','Dat','zeit','uhr']
new_list= [s for s in list if not any([x.lower for x in zeiten if x in s.lower()])]
new_list = [s for s in list if not df[s].isnull().values.all()]
new_list= [s for s in list if not (df[s] == '0.0').all()] 
new_list= [s for s in list if not (df[s] == 0.0).all()] 
new_list= [s for s in list if not (df[s] == 0).all()] 
new_list= [s for s in list if not (df[s] == 'nan').all()] 
new_list

['Buchungskreis',
 'Belegnummer',
 'Geschäftsjahr',
 'Belegart',
 'Buchungsperiode',
 'Erfassungsuhrzeit',
 'Geändert am',
 'Letzte Fortschreib',
 'Transaktionscode',
 'Übergreifende Nr',
 'Referenz',
 'Belegkopftext',
 'Währung',
 'Umrechnungskurs',
 'Konzernwährung',
 'Konzernkurs',
 'Belegstatus',
 'Dokumentenname im Archivsystem']

In [8]:
for column in new_list:
    try:
        if any(round(df[column].astype(float)) != df[column].astype(float)):
            print('float: ',column)
            df[column]=df[column].astype(float)
        elif all(round(df[column].astype(float)) == df[column].astype(float)):
            print('int: ',column)
            df[column]=df[column].astype(int).round()
    except:
        print('STR: ',column)

        #pass

int:  Buchungskreis
int:  Belegnummer
int:  Geschäftsjahr
STR:  Belegart
int:  Buchungsperiode
STR:  Erfassungsuhrzeit
float:  Geändert am
float:  Letzte Fortschreib
STR:  Transaktionscode
float:  Übergreifende Nr
float:  Referenz
float:  Belegkopftext
STR:  Währung
float:  Umrechnungskurs
float:  Konzernwährung
float:  Konzernkurs
float:  Belegstatus
float:  Dokumentenname im Archivsystem


In [81]:
pbjekts=pd.DataFrame(df.dtypes,columns=['types'])
pbjekts['unique']=df.nunique()
pbjekts['null']=df.isna().sum()
pbjekts['not_null']=df.count()
pbjekts
pr=pbjekts[pbjekts['unique']<200]
pg=pbjekts[pbjekts['unique']>=200]

In [87]:
from bokeh.plotting import figure, output_notebook, show,output_notebook
from bokeh.models import ColumnDataSource,HoverTool
from bokeh.transform import dodge

langs = pg.index.to_list()
data = {'langs' : langs,
        'unique'   : [ int(x) for x in pg['unique'].to_list() ]
        }
source = ColumnDataSource(data=data)

    
students = [ int(x) for x in pg['unique'].to_list() ]
fig = figure(x_range = langs, plot_width = 500, plot_height =400,toolbar_location=None, tools="", title="Unique values with qual or more than 200  unique values")
fig.xaxis.major_label_orientation = "vertical"
fig.vbar(x = dodge('langs',  0, range=fig.x_range), top='unique', width = 0.5,name='unique',source=source)

# fig.vbar(x=dodge('NullNotNull',  0.15, range=fig.x_range), top='notnull', width=0.2, source=source,
#        color="#e84d60", legend_label="notnull",name='notnull')
hover = HoverTool()
hover.tooltips = """
<div>
    <div><strong>Column:  </strong>@langs</div>
    <div><strong>Count: </strong>@$name</div>
</div>"""
fig.add_tools(hover)
output_notebook()
show(fig)

Loading BokehJS ...

In [86]:
from bokeh.plotting import figure, output_notebook, show,output_notebook
from bokeh.models import ColumnDataSource,HoverTool
from bokeh.transform import dodge

langs = pr.index.to_list()
data = {'langs' : langs,
        'unique'   : [ int(x) for x in pr['unique'].to_list() ]
        }
source = ColumnDataSource(data=data)

    
students = [ int(x) for x in pr['unique'].to_list() ]
fig = figure(x_range = langs, plot_width = 500, plot_height =400,toolbar_location=None, tools="", title="Unique values with under 200 unique values")
fig.xaxis.major_label_orientation = "vertical"
fig.vbar(x = dodge('langs',  0, range=fig.x_range), top='unique', width = 0.5,name='unique',source=source)

# fig.vbar(x=dodge('NullNotNull',  0.15, range=fig.x_range), top='notnull', width=0.2, source=source,
#        color="#e84d60", legend_label="notnull",name='notnull')
hover = HoverTool()
hover.tooltips = """
<div>
    <div><strong>Column:  </strong>@langs</div>
    <div><strong>Count: </strong>@$name</div>
</div>"""
fig.add_tools(hover)
output_notebook()
show(fig)

Loading BokehJS ...

In [65]:
from bokeh.plotting import figure, output_notebook, show,output_notebook
from bokeh.models import ColumnDataSource,HoverTool
from bokeh.transform import dodge

NullNotNull = pbjekts.index.to_list()
years = ['null', 'notnull']

data = {'NullNotNull' : NullNotNull,
        'null'   : [ int(x) for x in pbjekts['null'].to_list() ],
        'notnull'   : [ int(x) for x in pbjekts['not_null'].to_list() ]}

source = ColumnDataSource(data=data)


In [66]:
#fig = figure(x_range = NullNotNull, plot_width = 300, plot_height = 300)
fig = figure(x_range=NullNotNull, height=300, plot_height = 300, title="Null vs NotNull",
           toolbar_location=None, tools="")

fig.xaxis.major_label_orientation = "vertical"


fig.vbar(x=dodge('NullNotNull',  -0.15,  range=fig.x_range), top='null', width=0.2, source=source,
       color="#718dbf", legend_label="null",name='null')

fig.vbar(x=dodge('NullNotNull',  0.15, range=fig.x_range), top='notnull', width=0.2, source=source,
       color="#e84d60", legend_label="notnull",name='notnull')
hover = HoverTool()
hover.tooltips = """
<div>
    <div><strong>Column:  </strong>@NullNotNull</div>
    <div><strong>Count: </strong>@$name</div>
</div>"""
fig.add_tools(hover)
output_notebook()
show(fig)

Loading BokehJS ...

In [11]:
df.describe()


,Buchungskreis,Belegnummer,Geschäftsjahr,Buchungsperiode,Geändert am,Letzte Fortschreib,Übergreifende Nr,Referenz,Belegkopftext,Umrechnungskurs,Konzernwährung,Konzernkurs,Belegstatus,Dokumentenname im Archivsystem
count,2500.0,2.500000e+03,2500.0,2500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1710.0,8.595655e+08,2016.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,0.0,8.453429e+08,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1710.0,1.000021e+08,2016.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1710.0,1.000063e+08,2016.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1710.0,1.000105e+08,2016.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1710.0,1.800003e+09,2016.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1710.0,1.800009e+09,2016.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_string.describe()

,Buchungskreis,Belegnummer,Geschäftsjahr,Belegart,Belegdatum,Buchungsdatum,Buchungsperiode,Erfassungsdatum,Erfassungsuhrzeit,Geändert am,...,Übergreifende Nr,Referenz,Jahr,Belegkopftext,Währung,Umrechnungskurs,Konzernwährung,Konzernkurs,Belegstatus,Dokumentenname im Archivsystem
count,2500,2500,2500,2500,2500,2500,2500,2500,2500,0,...,0,0,0,0,2500,0,0,0,0,0
unique,1,2500,1,2,20,20,1,1,596,0,...,0,0,0,0,1,0,0,0,0,0
top,1710,100002131,2016,SA,2016-01-01 00:00:00,2016-01-01 00:00:00,1,2016-12-15 00:00:00,1900-01-01 11:53:01,NaN,...,NaN,NaN,NaN,NaN,USD,NaN,NaN,NaN,NaN,NaN
freq,2500,1,2500,1383,144,144,2500,2500,10,NaN,...,NaN,NaN,NaN,NaN,2500,NaN,NaN,NaN,NaN,NaN


In [13]:
column_data=df_string.describe()['count':'unique'].transpose()
column_data=column_data[column_data['count']!=0]
column_data_für_kar=column_data[column_data['unique']<=10]
kar_list=column_data_für_kar.index.to_list()

In [33]:
from turtle import backward


def anteil_null_values(df,column):
    null=df[column].isna().sum()
    notnull=df[column].count()
    explode=(0, 0.1)
    fig1, ax1 = plt.subplots()
    plt.title(f"Percentage of null values of {column}")
    ax1.pie([null,notnull], explode=explode, labels=['Null Values', 'Not Null Values'], autopct='%1.1f%%',
        shadow=True, startangle=90,)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    plt.show()

In [42]:
from bokeh.plotting import figure, output_notebook, show,output_notebook

values=df.describe()
values
langs = values.index.to_list()
students = [ int(x) for x in values['Buchungskreis'].to_list() ]
fig = figure(x_range = langs, plot_width = 300, plot_height = 300)
fig.xaxis.major_label_orientation = "vertical"
fig.vbar(x = langs, top = students, width = 0.5)
output_notebook()
show(fig)

Loading BokehJS ...

In [37]:
from bokeh.plotting import figure, output_notebook, show,output_notebook
values=pd.DataFrame(df_string['Buchungskreis'].value_counts()).reset_index(drop=False)
values

,index,Buchungskreis
0,1710,2500


In [40]:

langs = values['index'].to_list()
students = [ int(x) for x in values['Buchungskreis'].to_list() ]
fig = figure(x_range = langs, plot_width = 300, plot_height = 300)
fig.xaxis.major_label_orientation = "vertical"
fig.vbar(x = langs, top = students, width = 0.5)
output_notebook()
show(fig)

Loading BokehJS ...

In [26]:
from math import pi

import pandas as pd

from bokeh.io import output_file, show
from bokeh.palettes import BuGn,PRGn
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.plotting import figure, output_notebook, show


# ax1.pie(types_df['types'], explode=explode, labels=types_df['index'], autopct='%1.1f%%', pctdistance=0.7,labeldistance=1.05,radius=2000, 
#     shadow=True, startangle=90)
types_df=pd.DataFrame(pbjekts['types'].value_counts()).reset_index(drop=False)
types_df

,index,types
0,float64,10
1,datetime64[ns],5
2,int32,4
3,object,4


In [27]:
types_df['types']
data= types_df.copy()
data=data.rename(columns={'index':'type','types':'number'})
#z=data.set_index('type').to_dict()['number']
studentDict = dict(zip(data.type.astype(str), data.number))
data_fix = pd.Series(studentDict).reset_index(name='value').rename(columns={'index': 'country'})


In [29]:
data_fix['angle'] = data_fix['value']/data_fix['value'].sum() * 2*pi
data_fix

,country,value,angle
0,float64,10,2.731820
1,datetime64[ns],5,1.365910
2,int32,4,1.092728
3,object,4,1.092728


In [31]:
data_fix['color'] = BuGn[len(studentDict)]
data_fix

,country,value,angle,color
0,float64,10,2.731820,#238b45
1,datetime64[ns],5,1.365910,#66c2a4
2,int32,4,1.092728,#b2e2e2
3,object,4,1.092728,#edf8fb


In [32]:


p = figure(height=350, title="Pie Chart", toolbar_location=None,
           tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='country', source=data_fix)

p.axis.axis_label = None
p.axis.visible = False
p.grid.grid_line_color = None

show(p)

In [ ]:
pd.DataFrame(df['Buchungskreis'].describe()).reset_index(drop=False)